In [1]:
import unittest
from LogicaGioco import *

1

In [2]:
class TestCharacterFactory(unittest.TestCase):

    def setUp(self):
        self.creator1 = Player1Creator()
        self.creator2 = Player2Creator()

    # --- TEST DI CREAZIONE (Questi andavano già bene) ---
    def test_player1_creation(self):
        player = self.creator1.create_character("Giada", 0)
        self.assertIsInstance(player, Player1)
        self.assertEqual(player.nome, "Giada")
        self.assertEqual(player.moralita, 0)

    def test_player2_creation(self):
        player = self.creator2.create_character("Luca", 10)
        self.assertIsInstance(player, Player2)
        self.assertEqual(player.nome, "Luca")
        self.assertEqual(player.moralita, 10)

    def test_players_are_distinct(self):
        p1 = self.creator1.create_character("A", 0)
        p2 = self.creator2.create_character("A", 0)
        self.assertNotEqual(type(p1), type(p2))

    def test_multiple_instances(self):
        p1 = self.creator1.create_character("X", 10)
        p2 = self.creator1.create_character("Y", 0)
        self.assertNotEqual(p1, p2)
    
    def test_altruista_incrementa_correttamente(self): 
        player = self.creator1.create_character("Giada", 50) 
        # Nella nuova logica, passiamo la stringa come secondo argomento
        assegna_moralita(player, "eroe altruista") 
        self.assertEqual(player.moralita, 58) 

    def test_egoista_incrementa_correttamente(self): 
        player = self.creator1.create_character("Giada", 50) 
        assegna_moralita(player, "mercenario egoista") 
        self.assertEqual(player.moralita, 53) 
        
    def test_indifferente_incrementa_correttamente(self): 
        player = self.creator1.create_character("Giada", 50) 
        assegna_moralita(player, "anima indifferente") 
        self.assertEqual(player.moralita, 55)

    def test_nome_valido(self):
        # Simuliamo che l'utente abbia scritto "Giada " con uno spazio
        nome_pulito = valida_nome("Giada ", 1)
        self.assertEqual(nome_pulito, "Giada")

    def test_nome_default_player1(self):
        # Simuliamo invio vuoto per player 1
        nome_pulito = valida_nome("", 1)
        self.assertEqual(nome_pulito, "Player1")

    def test_nome_default_player2(self):
        # Simuliamo invio vuoto per player 2
        nome_pulito = valida_nome("   ", 2) # Anche stringa di soli spazi
        self.assertEqual(nome_pulito, "Player2")
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.012s

OK


In [3]:
class TestGameManager(unittest.TestCase):

    def setUp(self):
        """
        Garantisce l'isolamento: Resettiamo l'istanza Singleton prima di ogni test.
        Questo ci permette di testare la creazione 'pulita' ogni volta.
        """

        GameManager._instance = None


    def test_singleton_is_unique(self):
        """Verifica la proprietà fondaùmentale: una sola istanza (come richiesto dal pattern)."""
        manager1 = GameManager.get_instance()
        manager2 = GameManager.get_instance()

        self.assertIs(manager1, manager2, "GameManager non è un Singleton: sono state create due istanze diverse.")


    def test_initial_state_values(self):
        """Verifica che la prima istanza abbia i valori iniziali corretti (da __init__)."""
        manager1 = GameManager.get_instance()
        # Verifica dei vincoli (basati sul tuo codice: Livello 1, 5 Vite)

        self.assertEqual(manager1.livello_corrente, 1)
        self.assertEqual(manager1.vite_rimanenti, 5)
        self.assertEqual(manager1.giocatori, [])


    def test_reset_data_works(self):
        """Verifica che il metodo resetGameData() ripristini i valori iniziali (1 e 5)."""
        manager1 = GameManager.get_instance()

        # 1. Modifica lo stato (simulando una partita in corso)
        manager1.livello_corrente = 10 # Cambia il livello
        manager1.vite_rimanenti = 2  # Perde vite
        manager1.giocatori.append("Falso Giocatore")

        # 2. Azione: Resetta i dati
        manager1.resetGameData()

        # 3. Verifica: Lo stato deve essere tornato ai valori iniziali (5 e 1)
        self.assertEqual(manager1.livello_corrente, 1)
        self.assertEqual(manager1.vite_rimanenti, 5)
        self.assertEqual(manager1.giocatori, [])



if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.............
----------------------------------------------------------------------
Ran 13 tests in 0.011s

OK


Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.


2

In [4]:
import unittest

class TestAutoSaveSystem(unittest.TestCase):

    def setUp(self):
        """Prepara un player e un observer puliti prima di ogni test."""
        self.player = Player1("Giada", 50)
        
        self.observer = AutoSaveObserver() 
        self.player.attach(self.observer)

    def test_memento_stores_state(self):
        """Test 15: Verifica che il Memento catturi lo stato esatto."""
        memento = self.player.save_state()
        state = memento.get_state()
        self.assertEqual(state['nome'], "Giada")
        self.assertEqual(state['moralita'], 50)

    def test_restore_state_from_memento(self):
        """Test 16: Verifica che il Player sappia ripristinarsi da un Memento."""
        fake_state = {"nome": "GiadaPassata", "moralita": 10}
        memento = CharacterMemento(fake_state)
        self.player.restore_state(memento)
        
        self.assertEqual(self.player.nome, "GiadaPassata")
        self.assertEqual(self.player.moralita, 10)

    def test_no_save_if_no_change(self):
        # Salvataggio iniziale
        self.player.moralita = 60  # Impostiamo stesso valore, non dovrebbe salvare
        if os.path.exists("salvataggio_gioco.json"):
            with open("salvataggio_gioco.json", "r") as f:
                dati_iniziali = f.read()
        else:
            dati_iniziali = None

        # Impostiamo di nuovo lo stesso valore
        self.player.moralita = 60  # Non dovrebbe generare nuovo salvataggio

        # Controlliamo il file
        if os.path.exists("salvataggio_gioco.json"):
            with open("salvataggio_gioco.json", "r") as f:
                dati_finali = f.read()
        else:
            dati_finali = None

        self.assertEqual(dati_iniziali, dati_finali, "Il salvataggio non deve cambiare se il valore è identico")

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

................
----------------------------------------------------------------------
Ran 16 tests in 0.028s

OK


Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.


3

In [5]:
class TestHealthSystem(unittest.TestCase):

    def setUp(self):
        """
        Prepara l'ambiente prima di ogni test.
        Crea un player standard con 100 di vita massima.
        """
        self.player = Player("TestSubject", 50) 
        # Assumiamo che il costruttore ora inizializzi anche self.health = 100

    def test_initial_health(self):
        """Test 20: Verifica stato iniziale"""
        self.assertEqual(self.player.hp, 100)
        self.assertEqual(self.player.max_hp, 100)

    def test_take_damage_logic(self):
        """Test 21: Verifica matematica del danno"""
        self.player.take_damage(20)
        self.assertEqual(self.player.hp, 80)

    def test_heal_logic(self):
        """Test 23: Verifica matematica della cura"""
        # Prima danneggio per avere spazio di cura
        self.player.take_damage(50) # Vita a 50
        
        self.player.heal(30)
        self.assertEqual(self.player.hp, 80)

    def test_health_max_cap(self):
        """Test 25: Limite massimo (Criterio 4)"""
        self.player.take_damage(10) # Vita a 90
        self.player.heal(1000) # Curo tantissimo
        
        # Non deve superare 100
        self.assertEqual(self.player.hp, 100)

    def test_health_min_zero(self):
        """Test 26: Limite minimo (Criterio 4)"""
        self.player.take_damage(9999) # Danno letale
        
        # Non deve andare sotto 0
        self.assertEqual(self.player.hp, 0)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....................
----------------------------------------------------------------------
Ran 21 tests in 0.028s

OK


Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.


4

In [6]:
class TestInventoryAndPlayer(unittest.TestCase):

    def test_add_item(self):
        """25 - Verifica che gli oggetti vengano aggiunti alla lista interna"""
        inventory = Inventory()
        spada = Item("Spada", "Attacco", 10)
        inventory.add_item(spada)
        self.assertIn(spada, inventory._items)

    def test_iter_connection(self):
        """26 - Verifica che l'Inventario restituisca un iteratore"""
        inventory = Inventory()
        it = iter(inventory)
        self.assertIsInstance(it, InventoryIterator)

    def test_iterator_next(self):
        """27 - Verifica che il cursore avanzi correttamente sugli elementi"""
        inventory = Inventory()
        inventory.add_item(Item("A", "Utility", 0))
        inventory.add_item(Item("B", "Utility", 0))
        it = iter(inventory)
        self.assertEqual(next(it).nome, "A")
        self.assertEqual(next(it).nome, "B")

    def test_stop_iteration(self):
        """28 - Verifica che l'iteratore si fermi alla fine della lista"""
        inventory = Inventory()
        inventory.add_item(Item("A", "Utility", 0))
        it = iter(inventory)
        next(it)  # primo elemento
        with self.assertRaises(StopIteration):
            next(it)  # fine lista

    def test_empty_inventory(self):
        """30 - Verifica che l'inventario non dia errori se aperto quando è vuoto"""
        inventory = Inventory()
        it = iter(inventory)
        with self.assertRaises(StopIteration):
            next(it)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..............

Log: Dati di gioco resettati.


............
----------------------------------------------------------------------
Ran 26 tests in 0.039s

OK


Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
